In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %%capture --no-stderr
# !pip install python-dotenv langchain_openai langchain-chroma pypdf langchain langchain_community

In [ ]:
# 환경변수 설정

In [1]:
# 라이브러리 불러오기
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# <2024 부동산 보고서 RAG 챗봇>
# 라이브러리 불러오기
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

In [3]:
# PDF 문서 로드 및 텍스트 분할
# loader = PyPDFLoader("/content/drive/MyDrive/langchain-tutorial/Ch02. RAG/Data/2024_KB_부동산_보고서_최종.pdf")
loader = PyPDFLoader("./Data/2024_KB_부동산_보고서_최종.pdf")
documents = loader.load()  # 문서 로드

# 텍스트 분할 설정: 청크 크기와 겹침 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# 분할된 청크 수
print('분할된 청크의 수:', len(chunks))

분할된 청크의 수: 135


In [4]:
# 임베딩 생성 및 Chroma 데이터베이스 저장
embedding_function = OpenAIEmbeddings()  # 임베딩 모델 설정

# persist_directory = "/content/drive/MyDrive/langchain-tutorial/Ch02. RAG/directory/chroma"
persist_directory = "./directory/chroma"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory=persist_directory  # 데이터베이스 저장 경로
)

In [5]:
print('문서의 수:', vectorstore._collection.count())

문서의 수: 135


In [6]:
# 검색 및 재정렬
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # 관련 문서 상위 3개 검색 설정

# 프롬프트 템플릿 설정: 사용자 질문에 대한 답변을 생성하기 위한 템플릿
template = """당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.

컨텍스트: {context}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{question}")
    ]
)  # 템플릿 초기화
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)  # AI 모델 설정

In [7]:
print(prompt.format(context="컨텍스트 예시", chat_history=["대화 기록 예시1", "대화 기록 예시2"], question="질문 예시"))

System: 당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.

컨텍스트: 컨텍스트 예시

Human: 대화 기록 예시1
Human: 대화 기록 예시2
Human: 질문 예시


In [8]:
# 문서 형식 변환 함수 정의
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)  # 문서 내용을 줄바꿈으로 연결

In [9]:
# 체인 구성: 검색한 문서를 프롬프트에 연결하고 모델을 통해 응답 생성
chain = (
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.invoke(x["question"]))
    )  # 검색된 문서를 연결하여 전달
    | prompt
    | model
    | StrOutputParser()  # 결과를 문자열로 변환
)

In [10]:
# 대화 기록을 유지하기 위한 메모리 설정
chat_history = ChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,  # 세션 ID별 대화 기록 생성
    input_messages_key="question",
    history_messages_key="chat_history",
)

In [11]:
# 챗봇 실행 함수 정의
def chat_with_bot():
    session_id = "user_session"
    print("KB 부동산 보고서 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)")
    while True:
        user_input = input("사용자: ")
        if user_input.lower() == 'quit':
            break

        response = chain_with_memory.invoke(
            {"question": user_input},
            {"configurable": {"session_id": session_id}}
        )

        print("챗봇:", response)

# 메인 실행
if __name__ == "__main__":
    chat_with_bot()

KB 부동산 보고서 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)
챗봇: 랭체인 챗봇을 만들기 위한 실습을 진행 중이시군요! 랭체인(LLM Chain)은 대규모 언어 모델을 활용하여 다양한 작업을 수행할 수 있는 프레임워크입니다. 챗봇을 만들기 위해서는 다음과 같은 단계가 필요합니다:

1. **환경 설정**: Python과 필요한 라이브러리(예: LangChain, OpenAI API 등)를 설치합니다.

2. **API 키 설정**: OpenAI와 같은 언어 모델 API의 키를 설정합니다.

3. **기본 구조 설계**: 챗봇의 기본 구조를 설계합니다. 사용자 입력을 받고, 모델에 전달하여 응답을 생성하는 흐름을 만듭니다.

4. **대화 흐름 정의**: 사용자의 질문에 대한 응답을 어떻게 처리할지 정의합니다. 예를 들어, 특정 주제에 대한 질문에 대해 미리 정의된 답변을 제공할 수 있습니다.

5. **테스트 및 개선**: 챗봇을 테스트하고, 사용자 피드백을 바탕으로 개선합니다.

6. **배포**: 챗봇을 웹사이트나 애플리케이션에 통합하여 사용자들이 사용할 수 있도록 배포합니다.

구체적인 코드나 예제가 필요하시다면, 어떤 부분에 대해 더 알고 싶으신지 말씀해 주시면 도와드리겠습니다!
챗봇: 사용자님께서 "랭체인 챗못 만들기에 대해 실습중입니다"라고 말씀하셨습니다. 랭체인 챗봇을 만들기 위한 실습을 진행 중이라고 하셨습니다. 추가로 궁금한 점이나 도움이 필요하신 부분이 있으시면 말씀해 주세요!
챗봇: 2024 KB 부동산 보고서에 따르면, 수도권 주택시장은 전반적으로 침체 상태에 있으며, 특히 강남권과 같은 선호 지역에서는 상대적으로 강세를 보이고 있습니다. 다음은 수도권 주택 매매 전망에 대한 주요 내용입니다:

1. **거래 회복의 어려움**: 높은 기준금리와 주택 매매가격, DSR(총부채상환비율) 규제 등으로 인해 매수자들의 구매 여력이 회복되지 않고 있습니다. 매도자와 매수자 간의 희망가격 차이가 여전히 존재하여 거래가 위축되고 있

In [ ]:
# <스트림릿 적용>

# 라이브러리 설치
# %%capture --no-stderr
# !pip install streamlit pyngrok

# ngrok 인증키 설정
# !ngrok config add-authtoken 2tnwXk5jQ5uWVBnwy3Ou8Mdmu8v_5eAtrJHxGZRiGGKqsqHsh

app.py 파일을 생성하고 아래 코드를 복사해서 붙여넣기
```
# 스트림릿앱 실행
streamlit run app.py
```

In [ ]:
# app.py
import os
import streamlit as st
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

# 환경 변수 로드
load_dotenv("/content/.env")
api_key = os.getenv("OPENAI_API_KEY")

# PDF 처리 함수
@st.cache_resource
def process_pdf():
    # loader = PyPDFLoader("/content/drive/MyDrive/langchain-tutorial/Ch02. RAG/Data/2024_KB_부동산_보고서_최종.pdf")
    loader = PyPDFLoader("./Data/2024_KB_부동산_보고서_최종.pdf")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return text_splitter.split_documents(documents)

# 벡터 스토어 초기화
@st.cache_resource
def initialize_vectorstore():
    chunks = process_pdf()
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    return Chroma.from_documents(chunks, embeddings)

# 체인 초기화
@st.cache_resource
def initialize_chain():
    vectorstore = initialize_vectorstore()
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    template = """당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.

    컨텍스트: {context}
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{question}")
    ])

    model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=api_key)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    base_chain = (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["question"]))
        )
        | prompt
        | model
        | StrOutputParser()
    )

    return RunnableWithMessageHistory(
        base_chain,
        lambda session_id: ChatMessageHistory(),
        input_messages_key="question",
        history_messages_key="chat_history",
    )

# Streamlit UI
def main():
    st.set_page_config(page_title="KB 부동산 보고서 챗봇", page_icon="🏠")
    st.title("🏠 KB 부동산 보고서 AI 어드바이저")
    st.caption("2024 KB 부동산 보고서 기반 질의응답 시스템")

    # 세션 상태 초기화
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # 채팅 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 처리
    if prompt := st.chat_input("부동산 관련 질문을 입력하세요"):
        # 사용자 메시지 표시
        with st.chat_message("user"):
            st.markdown(prompt)
        st.session_state.messages.append({"role": "user", "content": prompt})

        # 체인 초기화
        chain = initialize_chain()

        # AI 응답 생성
        with st.chat_message("assistant"):
            with st.spinner("답변 생성 중..."):
                response = chain.invoke(
                    {"question": prompt},
                    {"configurable": {"session_id": "streamlit_session"}}
                )
                st.markdown(response)

        st.session_state.messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
    main()


2025-05-18 12:41:48.455 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 12:41:48.455 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 12:41:48.529 
  command:

    streamlit run /Users/maui/Repository/personal/books/book-rag-master/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-18 12:41:48.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 12:41:48.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 12:41:48.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 12:41:48.529 Thread 'MainThread': missing ScriptRun

In [ ]:
# 터널링 및 실행
from pyngrok import ngrok

public_url = ngrok.connect(8501)  # Streamlit 기본 포트
print("앱 접속 URL:", public_url)
!streamlit run /content/app.py


앱 접속 URL: NgrokTunnel: "https://d628-34-23-186-167.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.186.167:8501

/content/app.py:12: LangChainDeprecationWarning: Importing ChatMessageHistory from langchain.memory is deprecated. Please replace deprecated imports:

>> from langchain.memory import ChatMessageHistory

with new imports of:

>> from langchain_community.chat_message_histories import ChatMessageHistory
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.memory import ChatMessageHistory
/content/app.py:12: LangChainDeprecationWarning: Importing ChatMessageHistory from langchain.memory is deprecated. Please replace deprecated imports:

>> from langchain.memory import ChatMessageHistory

with new imports of:


  Stopping...
  Stopping...
